In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
# from google.colab import drive
# drive.mount('/content/drive')
import pandas as pd
from torchvision.io import decode_image
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets

Define Constants

In [4]:
labels_map = {
    0: "Anthracnose",
    1: "Banana Fruit-Scarring Beetle",
    2: "Banana Skipper Damage",
    3: 'Banana Split Peel',
    4: "Black and Yellow Sigatoka",
    5: "Chewing insect damage on banana leaf",
    6: "Healthy Banana",
    7: "Healthy Banana leaf",
    8: "Panama Wilt Disease",
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Define HyperParameters

In [3]:
num_classes = 9
# hyperparams
num_epochs = 1
batch_size = 64
learning_rate = 0.01
decay = 0.001
moment = 0.9

Import image dataset from drive

In [4]:

dataset_path = '/content/drive/MyDrive/bananadata/AUGMENTED/data'
build_path = '/content/drive/MyDrive/bananadata/build/augs'


Define Image transformations to fit model

In [5]:
normalize = transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5],
    )
transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize
    ])


Get images from drive and apply Transfomation to image datasets

In [6]:
dataset_train = datasets.ImageFolder(root=dataset_path+'/train', transform=transform)
dataset_val = datasets.ImageFolder(root=dataset_path+'/validation', transform=transform)
dataset_test = datasets.ImageFolder(root=dataset_path+'/test', transform=transform)
build_train = datasets.ImageFolder(root=build_path, transform=transform)

load images to DataLoader

In [8]:

load_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
load_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
load_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
load_build = DataLoader(build_train, batch_size=batch_size, shuffle=True)

In [9]:
load_train.dataset

Dataset ImageFolder
    Number of datapoints: 7605
    Root location: /content/drive/MyDrive/bananadata/AUGMENTED/data/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [10]:
for images, labels in load_val:
    print(images.shape)
    print(labels.shape)
    break

torch.Size([64, 3, 224, 224])
torch.Size([64])


Define VGG16 architecture CNN model

In [11]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

Define model

In [12]:
model = VGG16(num_classes).to(device)
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = decay, momentum = moment)


# Train the model
total_step = len(load_train)



### Training

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(load_train):
        # Move tensors to the configured device
        images = batch[0].to(device)
        labels = batch[1].to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(load_train.dataset)
    train_losses.append(train_loss)

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Validation
    running_loss = 0.0
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in load_val:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
            running_loss += loss.item() * labels.size(0)
        val_loss = running_loss / len(load_val.dataset)
        val_losses.append(val_loss)
        val_acc = 100 * correct / total
        val_accuracies.append(val_acc)
        print('Accuracy of the network on the {} validation images: {} %'.format(945, 100 * correct / total))


## PLOTS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

loss plots

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

In [ ]:

model.eval()
test_y_true = []
test_y_pred = []

with torch.no_grad():
    correct = 0
    total = 0
    for batch in load_test:
        images = batch['image'].to(device)
        labels = batch['label'].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

        del images, labels, outputs


In [ ]:
print('Accuracy of the network on the {} test images: {} %'.format(len(test_ds), 100 * correct / total))

cm = confusion_matrix(y_true, y_pred)

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels_map.values(), yticklabels=labels_map.values())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

load build dataset here also augment it equally

maybe save augmented image dataset for easy repeatability

In [ ]:
model.eval()
build_y_true = []
build_y_pred = []

with torch.no_grad():
    correct = 0
    total = 0
    # change to buuild dataset
    for images, labels in load_build:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

In [ ]:
# prompt: save hyperparameters and results from the learning rate, epoch, batch size, losses and accuracies, test_y_pred and test_y_true, build_y_true and build_y_pred

import json
import os

# Define the directory to save the results
results_dir = "/content/drive/MyDrive/bananadata/results/cnn"
os.makedirs(results_dir, exist_ok=True)

# Define hyperparameters and results
hyperparameters = {
    "learning_rate": learning_rate,
    "epoch": num_epochs,
    "batch_size": batch_size,
    "optimizer": "SGD",
    "loss_function": "CrossEntropyLoss",
    "model_architecture": "VGG16"
}

results = {
    "train_losses": train_losses,
    "val_losses": val_losses,
    # Assuming you calculated training and validation accuracies and stored them in train_accuracies and val_accuracies lists
    "val_accuracies": val_accuracies,
    "test_y_true": test_y_true,
    "test_y_pred": test_y_pred,
    "build_y_true": build_y_true,
    "build_y_pred": build_y_pred,
}

test_index = 1
# Save hyperparameters
hyperparameters_path = os.path.join(results_dir, f"hyperparameters{test_index}.json")
with open(hyperparameters_path, "w") as f:
    json.dump(hyperparameters, f, indent=4)
print(f"Hyperparameters saved to {hyperparameters_path}")

# Save results
results_path = os.path.join(results_dir, f"results{test_index}.json")
with open(results_path, "w") as f:
    json.dump(results, f)
print(f"Results saved to {results_path}")